<a href="https://colab.research.google.com/github/danjohnvelasco/STADVDB_S13.2_T120-21/blob/main_mco2/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCO2
**Instructions to run:**

Just run all the cells (ctrl + F9) and you'll be fine 😉

**Dataset:** Brazilian E-Commerce Public Dataset by Olist [[Kaggle]](https://www.kaggle.com/olistbr/brazilian-ecommerce) [[download here](https://drive.google.com/file/d/1gUKUkb49cCVEM7YuqtMjNeVk_x8vUDBe/view?usp=sharing)]

**Tables to be used from dataset (6 out of 9):**
- Customers: customer, location
- Order Items: items purchased **within** each order
- Order: order info (purchase date, delivery date, status)
- Products: products sold by Olist
- Sellers: sellers that fulfilled orders made
- Category Name: english translation of `productcategoryname`


---



#### Fact Table Schema:

**Dimensions:**
- order_approved_at
- order_id
- customer_id
- seller_id
- product_id

**Measures:**
- unit_sales (total items within each order)
- sales_amount (total amount within each order)


In [1]:
import pandas as pd

## Extract

Loading data to tool

In [2]:
!gdown --id 1gUKUkb49cCVEM7YuqtMjNeVk_x8vUDBe

Downloading...
From: https://drive.google.com/uc?id=1gUKUkb49cCVEM7YuqtMjNeVk_x8vUDBe
To: /content/olist_dataset.zip
44.8MB [00:00, 139MB/s] 


In [3]:
!mkdir data
!unzip olist_dataset.zip -d data && rm olist_dataset.zip
%cd data

mkdir: cannot create directory ‘data’: File exists
Archive:  olist_dataset.zip
replace data/olist_customers_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/olist_customers_dataset.csv  
  inflating: data/olist_geolocation_dataset.csv  
  inflating: data/olist_order_items_dataset.csv  
  inflating: data/olist_order_payments_dataset.csv  
  inflating: data/olist_order_reviews_dataset.csv  
  inflating: data/olist_orders_dataset.csv  
  inflating: data/olist_products_dataset.csv  
  inflating: data/olist_sellers_dataset.csv  
  inflating: data/product_category_name_translation.csv  
/content/data


In [4]:
customers = pd.read_csv('olist_customers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
english_categories = pd.read_csv('product_category_name_translation.csv')

## Transform


#### Products Table

Drop rows with NaN at `product_category_name`


In [5]:
# Get indexes of rows with NaNs at column product_category_name
idx_cat_nan = products.loc[products['product_category_name'].isnull() == True ].index

# Drop rows
products.drop(idx_cat_nan, inplace=True)

Translate product_category_name to english

In [6]:
products = pd.merge(products, english_categories, on='product_category_name')
products['product_category_name'] = products['product_category_name_english']
products.drop(['product_category_name_english'], axis=1, inplace=True)
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,39.0,346.0,2.0,400.0,27.0,5.0,20.0
2,0d009643171aee696f4733340bc2fdd0,perfumery,52.0,150.0,1.0,422.0,21.0,16.0,18.0
3,b1eae565a61935e0011ee7682fef9dc9,perfumery,49.0,460.0,2.0,267.0,17.0,13.0,17.0
4,8da90b37f0fb171b4877c124f965b1f6,perfumery,56.0,733.0,3.0,377.0,18.0,13.0,15.0


### Dimensionality Modeling


In [7]:
df = order_items.groupby(['order_id', 'product_id', 'seller_id'])['price'].agg(unit_sales='count', total_sales='sum').reset_index()
df.head()

,order_id,product_id,seller_id,unit_sales,total_sales
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,1,239.90
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,1,199.90


In [8]:
fact = pd.merge(df, orders[['order_id', 'customer_id', 'order_approved_at']], on='order_id')
fact = fact[['order_approved_at', 'order_id', 'customer_id', 'product_id', 'seller_id', 'unit_sales', 'total_sales']]
fact.head()

,order_approved_at,order_id,customer_id,product_id,seller_id,unit_sales,total_sales
0,2017-09-13 09:45:35,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,1,58.90
1,2017-04-26 11:05:13,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,1,239.90
2,2018-01-14 14:48:30,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1,199.00
3,2018-08-08 10:10:18,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,1,12.99
4,2017-02-04 14:10:13,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,1,199.90


## Load

Export to csv (will be loaded to MySQL later)

In [9]:
products.to_csv('products_transformed.csv', index=False)
fact.to_csv('fact.csv', index=False)

In [10]:
# Save data directory to google drive
%cd ..
!cp -r data "/content/drive/My Drive/advdb datawarehouse"

cp: cannot stat 'data': No such file or directory
